In [1]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='syp', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: syp...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [14]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [8]:
user = load_data_mart(device="console", start_date="2021-09-09", end_date="2021-10-20", table_name="user_master")

In [4]:
gcoin = mysql.read_table(spark, "pubg_gi", "all_economy_gcoin_used_by_product_daily")

In [9]:
items = mysql.read_table(spark, "metainfo", "meta_vc_sales_items")
pcs5_items = items.where(col("event_name") == "202109_pcs5").select("product_id", "product_name").distinct().toPandas()
pcs5_product_id_list = pcs5_items.product_id.tolist()

In [10]:
startdate = "2021-09-09"
enddate = "2021-09-15"
wau = user.where((col("date") >= startdate) & (col("date") <= enddate) & (col("lastlogindate") >= startdate)).select(countDistinct("accountid").alias("wau")).collect()[0][0]
gc = gcoin.where((col("device") == "CONSOLE") & (col("date") >= startdate) & (col("date") <= enddate) & (col("product_id") == "total")).select((sum("free_gcoin_used")/100).alias("free"), (sum("paid_gcoin_used")/100).alias("paid"), ((sum(col("paid_gcoin_used")+col("free_gcoin_used"))/100).alias("total"))).collect()

In [11]:
print(startdate + "~" + enddate)
print("wau", wau)
print("free", gc[0][0])
print("paid", gc[0][1])
print("total", gc[0][2])

2021-09-09~2021-09-15
wau 426810
free 53971.8
paid 326463.8
total 380435.6


In [26]:
startdates = ["2021-09-16", "2021-09-23", "2021-09-30", "2021-10-07", "2021-10-14"]
enddates = ["2021-09-22", "2021-09-29", "2021-10-06", "2021-10-13", "2021-10-20"]
wau_list = []
free_gcoin = []
paid_gcoin = []
total_gcoin = []

In [28]:
for i in range(len(startdates)):
    startdate = startdates[i]
    enddate = enddates[i]
    print(startdate + " ~ " + enddate)
    wau = user.where((col("date") >= startdate) & (col("date") <= enddate) & (col("lastlogindate") >= startdate)).select(countDistinct("accountid").alias("wau")).collect()[0][0]
    wau_list.append(wau)
    gc = gcoin.where((col("device") == "CONSOLE") & (col("date") >= startdate) & (col("date") <= enddate) & (col("product_id").isin(pcs5_product_id_list))).select((sum("free_gcoin_used")/100).alias("free"), (sum("paid_gcoin_used")/100).alias("paid"), ((sum(col("paid_gcoin_used")+col("free_gcoin_used"))/100).alias("total"))).collect()
    free_gcoin.append(gc[0][0])
    paid_gcoin.append(gc[0][1])
    total_gcoin.append(gc[0][2])

2021-09-16~2021-09-22
2021-09-23~2021-09-29
2021-09-30~2021-10-06
2021-10-07~2021-10-13
2021-10-14~2021-10-20


In [31]:
d = {
    "start_date": startdates,
    "end_date": enddates,
    "wau": wau_list,
    "free": free_gcoin,
    "paid": paid_gcoin,
    "total": total_gcoin
}
df = pd.DataFrame.from_dict(d)
df.to_csv("./data/console_arpu.csv", index=False)

In [32]:
df

,start_date,end_date,wau,free,paid,total
0,2021-09-16,2021-09-22,417293,23567.2,228960.4,252527.6
1,2021-09-23,2021-09-29,395959,4715.8,48933.8,53649.6
2,2021-09-30,2021-10-06,392569,3394.6,37104.0,40498.6
3,2021-10-07,2021-10-13,382593,2573.4,29287.4,31860.8
4,2021-10-14,2021-10-20,354577,3246.6,30987.4,34234.0


In [5]:
user2 = load_data_mart(device="console", start_date="2022-02-17", end_date="2022-02-23", table_name="user_master")

In [6]:
user2.where((col("lastlogindate") >= "2022-02-17")).select(countDistinct("accountid")).show()

+-------------------------+
|count(DISTINCT accountid)|
+-------------------------+
|                  1067741|
+-------------------------+



In [7]:
gcoin.where((col("device") == "CONSOLE") & (col("date") >= "2022-02-17") & (col("date") <= "2022-02-23") & (col("product_id") == "total")).select((sum("free_gcoin_used")/100).alias("free"), (sum("paid_gcoin_used")/100).alias("paid"), ((sum(col("paid_gcoin_used")+col("free_gcoin_used"))/100).alias("total"))).toPandas()

,free,paid,total
0,893567.2,376649.4,1270216.6
